# Pomer dát 7:3

Súbor: train_7_test_3.ipynb

Program: Hospodárska informatika

Vypracovala:  Veronika Motúzová 

Bakalárska práca: Metódy hlbokého učenia pre detekciu meteorov v astronomických snímkach

Vedúci bakalárskej práce: doc. Ing. Peter Butka, PhD.

Konzultant: Ing. Viera Maslej Krešňáková

Import knižníc

In [9]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.layers import Conv2D, MaxPooling2D, Dropout, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
import array
from sklearn.metrics import confusion_matrix,classification_report
import itertools 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 
from sklearn.model_selection import train_test_split

Načítanie dát

In [2]:
train = np.load('../data/data_final_train.npz')
test = np.load('../data/data_final_test.npz')

In [3]:
x_train = train['train_data'].reshape(train['train_data'].shape[0], 128, 128, 1)
x_test = test['test_data'].reshape(test['test_data'].shape[0], 128, 128, 1)

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [5]:
y_train = np_utils.to_categorical(train['train_targets'], 2)
y_test = np_utils.to_categorical(test['test_targets'], 2)

In [6]:
X = np.concatenate([x_train,x_test])

In [7]:
Y = np.concatenate([y_train,y_test])

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.3)

In [12]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(33244, 128, 128, 1)
(14248, 128, 128, 1)
(33244, 2)
(14248, 2)


Začíname vrstviť neurónku AlexNet Architecture

In [13]:
model = Sequential()

In [14]:
model.add(ZeroPadding2D((1,1),input_shape=(128, 128,1)))

In [15]:
model.add(Conv2D(64, 3, strides=1, activation='relu')) 

In [16]:
model.add(MaxPooling2D(3, 2))

In [17]:
model.add(Conv2D(64, 5, strides=1, padding='same', activation='relu'))

In [18]:
model.add(MaxPooling2D(3, 2))

In [19]:
model.add(Conv2D(64, 3, strides=1, padding='same', activation='relu'))

In [20]:
model.add(Conv2D(64, 3, strides=1, padding='same', activation='relu'))

In [21]:
model.add(MaxPooling2D(3, 2))

In [22]:
model.add(Flatten())

In [23]:
model.add(Dense(96, activation='relu'))

In [24]:
model.add(Dropout(0.25)) 

In [25]:
model.add(Dense(96, activation='relu'))

In [26]:
model.add(Dropout(0.25)) 

In [27]:
model.add(Dense(2, activation='softmax'))

Adam Optimization

In [28]:
adam = Adam()

In [29]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Aby sa uložil model s najlepším výsledkom:

In [30]:
saved_model = "model_7_3.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

test loss je chyba klasifikácie (FN + FP) / súčet všetkých

test accuracy je úspešnosť klasifikácie (TP + TN) / súčet všetkých

In [31]:
f = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test), callbacks=[checkpoint])


Train on 33244 samples, validate on 14248 samples
Epoch 1/20
33244/33244 [==============================] - 1809s 54ms/step - loss: 0.3457 - accuracy: 0.8452 - val_loss: 0.2056 - val_accuracy: 0.9294

Epoch 00001: val_accuracy improved from -inf to 0.92939, saving model to model_4.hdf5
Epoch 2/20
33244/33244 [==============================] - 1606s 48ms/step - loss: 0.2139 - accuracy: 0.9201 - val_loss: 0.2243 - val_accuracy: 0.9116

Epoch 00002: val_accuracy did not improve from 0.92939
Epoch 3/20
33244/33244 [==============================] - 1587s 48ms/step - loss: 0.1765 - accuracy: 0.9363 - val_loss: 0.1952 - val_accuracy: 0.8997

Epoch 00003: val_accuracy did not improve from 0.92939
Epoch 4/20
33244/33244 [==============================] - 1588s 48ms/step - loss: 0.1625 - accuracy: 0.9434 - val_loss: 0.1628 - val_accuracy: 0.9406

Epoch 00004: val_accuracy improved from 0.92939 to 0.94062, saving model to model_4.hdf5
Epoch 5/20
33244/33244 [==============================] - 15

Načítanie modelu

In [34]:
print("Loading model....")
model = load_model('model_7_3.hdf5')

Loading model....


Vyhodnotenie test loss a accuracy

In [40]:
#na trénovacej množine 
score = model.evaluate(x_train, y_train, verbose=0) 
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02440297023264682
Test accuracy: 0.9909757971763611


In [41]:
#na testovacej množine
score1 = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

Test loss: 0.08178553220754252
Test accuracy: 0.9809798002243042


Predikcia

In [36]:
y_pred = model.predict(x_test)
prediction_int = np.zeros_like(y_pred)
prediction_int[y_pred > 0.5] = 1

Confusion Matrix

In [37]:
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [38]:
# TN / FN
# FP / TP
print(matrix)

[[7095   57]
 [ 214 6882]]


Classification Report

In [39]:
print(classification_report(y_test, prediction_int))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7152
           1       0.99      0.97      0.98      7096

   micro avg       0.98      0.98      0.98     14248
   macro avg       0.98      0.98      0.98     14248
weighted avg       0.98      0.98      0.98     14248
 samples avg       0.98      0.98      0.98     14248



In [35]:
model.summary() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 31, 31, 64)       